# Convert a Saved Model to TF-Lite Model

### Import all the Dependencies

In [2]:
import numpy as np
import pickle

import os
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
import sklearn

### Set all the Constants

In [3]:
INIT_LR = 1e-3
BATCH_SIZE = 32
EPOCHS = 50
IMAGE_SIZE = 256
default_image_size = tuple((IMAGE_SIZE, IMAGE_SIZE))
image_size = 0
data_dir = "../../PlantVillage (copy)"
CHANNELS=3

### Split Dataset

Dataset should be bifurcated into 3 subsets, namely:
1. Training: Dataset to be used while training
2. Validation: Dataset to be tested against while training
3. Test: Dataset to be tested against after we trained a model

In [13]:
import os
print(os.path.exists("Plant Village_files"))  # This should return True if the directory exists
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "/Users/vineetpatel/VINEET SMIT/3rd year/5th sem/PBL/code/potato-disease-classification/dataset",
    seed=123,
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)


False
Found 2152 files belonging to 3 classes.


In [15]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "/Users/vineetpatel/VINEET SMIT/3rd year/5th sem/PBL/code/potato-disease-classification/dataset",
    seed=123,
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)


Found 2152 files belonging to 3 classes.


In [19]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "/Users/vineetpatel/VINEET SMIT/3rd year/5th sem/PBL/code/potato-disease-classification/dataset",
    seed=123,
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)
import os
print(os.getcwd())



Found 2152 files belonging to 3 classes.
/Users/vineetpatel/VINEET SMIT/3rd year/5th sem/PBL/code/potato-disease-classification/training


In [34]:

import tensorflow as tf

def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + val_split + test_split) == 1, "Splits must sum to 1"
    
    # Ensure that cardinality is a positive integer
    ds_size = ds.cardinality().numpy()
    
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    # Take train_size elements for training
    train_ds = ds.take(train_size)
    
    # Skip the training data and take val_size elements for validation
    val_ds = ds.skip(train_size).take(val_size)
    
    # Skip both training and validation data for test set
    test_ds = ds.skip(train_size + val_size)
    
    return train_ds, val_ds, test_ds

# Assuming data_dir, default_image_size, and BATCH_SIZE are defined
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    seed=123,
    image_size=default_image_size,
    batch_size=BATCH_SIZE
)

train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)


NotFoundError: Could not find directory /Users/yourname/datasets/potato_diseases

In [35]:


def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    
    ds_size = ds.cardinality().numpy()
    
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

dataset = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  seed=123,
  image_size=default_image_size,
  batch_size=BATCH_SIZE
)


train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

NotFoundError: Could not find directory /Users/yourname/datasets/potato_diseases

In [21]:
import os
print(os.path.exists(data_dir))  # This should return True if the path is correct
data_dir = "/path/to/your/dataset"
data_dir = "./dataset"
data_dir = "/Users/yourname/datasets/potato_diseases"
dataset = tf.keras.preprocessing.image_dataset_from_directory
(
  data_dir,
  seed=123,
  image_size=default_image_size,
  batch_size=BATCH_SIZE
)

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (2974251711.py, line 9)

In [22]:
class_names = ["Potato___Early_blight", "Potato___Late_blight", "Potato___healthy"]


### Convert the Saved Model to TF Lite Model

In [29]:
converter = tf.lite.TFLiteConverter.from_saved_model("/Users/vineetpatel/VINEET SMIT/3rd year/5th sem/PBL/code/potato-disease-classification/tf-lite-models")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

OSError: SavedModel file does not exist at: /Users/vineetpatel/VINEET SMIT/3rd year/5th sem/PBL/code/potato-disease-classification/tf-lite-models/{saved_model.pbtxt|saved_model.pb}

### Evaluate the TF Lite Model

In [30]:
def evaluate_tflite_model(dataset, interpreter):
    input_index = interpreter.get_input_details()[0]["index"]
    output_index = interpreter.get_output_details()[0]["index"]

    prediction_digits = []
    test_labels = []
    for image, label in dataset.unbatch().take(dataset.unbatch().cardinality()):

        test_image = np.expand_dims(image, axis=0).astype(np.float32)
        interpreter.set_tensor(input_index, test_image)
        interpreter.invoke()
        
        output = interpreter.tensor(output_index)
        digit = np.argmax(output()[0])
        prediction_digits.append(digit)
        test_labels.append(label)

    prediction_digits = np.array(prediction_digits)
    accuracy = (prediction_digits == test_labels).mean()
    return accuracy

interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

test_accuracy = evaluate_tflite_model(dataset, interpreter)

print('TFLite test_accuracy:', test_accuracy)



NameError: name 'tflite_model' is not defined

### Save the TF Lite Model

In [31]:
import os
model_version = max([int(i.split(".")[0]) for i in (os.listdir("../tf-lite-models")+["0.tflite"])]) + 1

with open(
    f"../tf-lite-models/{model_version}.tflite",
    'wb'
) as f:
    f.write(tflite_model)

NameError: name 'tflite_model' is not defined

### Plotting the Inference Data

In [32]:
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

plt.figure(figsize=(15, 15))
for images, labels in test_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))        

        actual_class = class_names[labels[i]]

        test_image = np.expand_dims(images[i], axis=0).astype(np.float32)
        interpreter.set_tensor(input_index, test_image)
        interpreter.invoke()
        output = interpreter.tensor(output_index)
        digit = np.argmax(output()[0])

        predicted_class = class_names[digit]
        confidence = np.max(output()[0])*100

        plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence}%")
        plt.axis("off")

NameError: name 'interpreter' is not defined